# Standard addition of SSDE

In [1]:
load('common_computations-ssde.sage', 'common_computations-standard-addition.sage')
var('q')

q

In [2]:
DigitwiseSum(q)

Transducer with 6 states

We will use the following sets, defined in the separate file common_computations.sage such that we can combine several transitions between two states:

In [3]:
digits_labels(q)

[-q,
 -H,
 -1/2*q - 1,
 -1/2*q,
 -1/2*q + 1,
 -L,
 0,
 L,
 1/2*q - 1,
 1/2*q,
 1/2*q + 1,
 H,
 q]

Now, we can construct the transducer to perform the addition:

In [4]:
def transformation(state, d, q):
    digit = digits(q)[d]
    if state == 'q/2':
        if digit == digits(q)[q]:
            new_carry = 1
            old_carry = 0
        elif greaterequal0(digit.min-q/2):
            new_carry = 1
            old_carry = 1
        elif greaterequal0(digit.min) and greater0(q/2-digit.max):
            new_carry = 0
            old_carry = 0
        elif greaterequal0(digit.min+q/2) and greater0(-digit.max):
            new_carry = 0
            old_carry = 1
        else:
            new_carry = -1
            old_carry = 0
        new_state = new_carry
        carry = [new_carry, old_carry]
    elif state == '-q/2':
        if digit == digits(q)[-q]:
            new_carry = -1
            old_carry = 0
        elif greaterequal0(-digit.max-q/2):
            new_carry = -1
            old_carry = -1
        elif greaterequal0(-digit.max) and greater0(digit.min+q/2):
            new_carry = 0
            old_carry = 0
        elif greaterequal0(-digit.max+q/2) and greater0(digit.min):
            new_carry = 0
            old_carry = -1
        else:
            new_carry = 1
            old_carry = 0
        new_state = new_carry
        carry = [new_carry, old_carry]
    else:
        current = digit + state
        if current == digits(q)[q/2]:
            carry = None
            return ('q/2', carry)
        elif current == digits(q)[-q/2]:
            carry = None
            return ('-q/2', carry)
        elif greater0(-q/2-current.max):
            carry = -1
        elif greater0(current.min-q/2):
            carry = 1
        else:
            carry = 0
        new_state = carry
    if carry is None:
        raise LookupError
    return (new_state, carry)
@cached_function
def Addition(q):
    def transformation_q(x, y):
        return transformation(x, y, q)
    return Transducer(transformation_q, 
    input_alphabet=digits_labels(q),
    initial_states=[0], final_states=[0])

In [5]:
Add_q = Addition(q)
Add_q.set_coordinates({0: (-2, 0), 1: (4, 4), -1: (4, -4), 'q/2': (0, 0), '-q/2': (-6, 0)})
#view(Add_q)

DigitwiseSum is the simplification of the weighted automaton $\mathcal{A}^2$ recognizing the digitwise sum of two SSDE.

In [6]:
DigitwiseSum(q)

Transducer with 6 states

Combining the these two finite state machine gives the weighted automaton S_SSDE.

As the order of the states may change, depending on the operating system, etc., the output may be a different permutation. However, the final results are not influenced.

In [7]:
sage.combinat.finite_state_machine.FSMOldProcessOutput = False
@cached_function
def S_SSDE(q):
    return Addition(q)(DigitwiseSum(q))
for s in S_SSDE(q).iter_states(): # not tested
    print s 

((((0, 0),),), 0)
((((0, 0),),), 1)
((((-1, -1),),), -1)
((((-1, -1),),), '-q/2')
((((-1, -1),),), 0)
((((-1, -1),),), 'q/2')
((((-1, -1),),), 1)
((((-1, 0), (0, -1)),), 0)
((((-1, 0), (0, -1)),), 'q/2')
((((-1, 0), (0, -1)),), 1)
((((-1, 1), (1, -1)),), 0)
((((-1, 1), (1, -1)),), 'q/2')
((((-1, 1), (1, -1)),), 1)
((((0, 1), (1, 0)),), 1)
((((-1, 0), (0, -1)),), -1)
((((-1, 0), (0, -1)),), '-q/2')
((((0, 1), (1, 0)),), 0)
((((0, 1), (1, 0)),), 'q/2')
((((1, 1),),), 1)
((((0, 0),),), '-q/2')
((((0, 0),),), 'q/2')
((((0, 0),),), -1)


In [8]:
var('x y')

(x, y)

In [9]:
def matrix_function(transition):
    result = transition.word_in[0] * x^add(list(j for j in transition.word_out if j==1)) \
                * y^(-add(list(j for j in transition.word_out if j==-1)))
    if type(result) == sage.structure.factorization.Factorization:
        result = expand(result).factor()
    return result
@cached_function
def A(q):
    return S_SSDE(q).adjacency_matrix(entry=matrix_function)

In [10]:
A_q = A(q)
for i in range(A(q).nrows()): # not tested
    for j in range(A_q.ncols()):
        entry = (A_q*8*q^2)[i, j]
        if entry:
            print entry.factor()
        else:
            print entry
    print "----" 

6*q^2 - 12*q + 8
(q - 2)*(q - 4)*x
2*y
0
0
0
0
4*q - 8
0
0
4
0
0
4*(q - 2)*x
4*(q - 2)*y
8
4*q - 8
8
2*x
4*q - 8
4*q - 8
(q - 2)*(q - 4)*y
----
6*(q - 2)*q
(q - 2)*q*x
2*y
0
0
0
0
4*q
0
0
4
0
0
4*q*x
4*(q - 4)*y
8
4*q - 16
8
2*x
4*q - 16
4*q
(q - 4)*(q - 6)*y
----
4*(q - 2)*q
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
16*q
0
4*(q - 2)*q*y
----
4*(q + 2)*q
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4*(q - 2)*q*y^2
----
4*(q + 2)*q
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
16*q - 32
0
4*(q - 2)*(q - 4)*y
----
4*q^2*x + 24*q*x - 64*x + 32
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4*(q - 2)*(q - 4)*y
----
4*q^2 + 24*q - 32
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
16*q - 64
0
4*(q - 4)*(q - 6)*y
----
2*(3*q - 2)*q
0
0
0
0
0
0
0
0
0
0
0
0
0
4*(q - 2)*y
8
4*q - 8
8
0
8*q - 16
0
2*(q - 2)*(q - 4)*y
----
2*(2*q*x + q - 4*x)*(q + 2)
0
0
0
0
0
0
8*x
0
0
0
0
0
8*x^2
4*(q - 2)*y
0
4*q - 8
0
0
0
0
2*(q - 2)*(q - 4)*y
----
2*(3*q + 8)*(q - 2)
0
0
0
0
0
0
8
0
0
0
0
0
8*x
4*(q - 4)*y
8
4*q - 16
8
0
8*q - 32
16
2*(q - 4)*(q -

Next, we construct the adjacency matrix and compute its determinant. For a better performance, we switch from the symbolic ring to a multivariate polynomial ring.

In [11]:
@cached_function
def sysM(q):
    var('z')
    return identity_matrix(ZZ,A(q).ncols()) - z*A(q)
@cached_function
def det_MQ(q):
    sysM_q = sysM(q)
    k=2
    RQ.<q,y,x,z> = QQ[]
    RQM = MatrixSpace(RQ, sysM_q.ncols())
    MQ = RQM((sysM_q*q^k).expand())
    return MQ.determinant() / q^(k*sysM_q.ncols())
@cached_function
def results(q):
    return expectation_variance(det_MQ(q))
results_q = results(q)
(results_q['expectation_x'].factor(), results_q['expectation_y'].factor())

((1/8) * (q + 1)^-2 * (q^2 + 2*q + 4), (1/8) * (q + 1)^-2 * (q^2 + 2*q + 4))

In [12]:
(results_q['variance_x'].factor(), results_q['variance_y'].factor(), results_q['covariance'].factor())

((1/64) * (q + 1)^-5 * (q - 1)^-1 * (7*q^6 + 48*q^5 + 159*q^4 + 128*q^3 - 48*q^2 - 12*q - 8),
 (1/64) * (q + 1)^-5 * (q - 1)^-1 * (7*q^6 + 48*q^5 + 159*q^4 + 128*q^3 - 48*q^2 - 12*q - 8),
 (-1/64) * (q + 1)^-5 * (q - 1)^-1 * (q^6 + 24*q^5 + 33*q^4 + 80*q^3 + 120*q^2 - 12*q - 8))

Determinant of the variance-covariance matrix, which is non-zero for all q and -q+1<d<0:

In [13]:
(results_q['variance_x']*results_q['variance_y'] - results_q['covariance']^2).factor()

(3/256) * (q + 1)^-6 * (q - 1)^-2 * q^2 * (q^2 + 5*q - 2) * (q^4 + 4*q^3 + 21*q^2 + 8*q - 28)

In [14]:
with open('table-var-standard-ssde', 'w') as f:
    f.write("q varx covar\n")
    for q1 in range(1, 50):
        f.write("%d %10f %10f\n" %(2*q1, results(q)['variance_x'].subs(q=2*q1), results(q)['covariance'].subs(q=2*q1)))

In [15]:
results(q)

{'covariance': (1/64*q^6 + 3/8*q^5 + 33/64*q^4 + 5/4*q^3 + 15/8*q^2 - 3/16*q - 1/8)/(-q^6 - 4*q^5 - 5*q^4 + 5*q^2 + 4*q + 1),
 'expectation_x': (-1/8*q^2 - 1/4*q - 1/2)/(-q^2 - 2*q - 1),
 'expectation_y': (-1/8*q^2 - 1/4*q - 1/2)/(-q^2 - 2*q - 1),
 'variance_x': (-7/64*q^6 - 3/4*q^5 - 159/64*q^4 - 2*q^3 + 3/4*q^2 + 3/16*q + 1/8)/(-q^6 - 4*q^5 - 5*q^4 + 5*q^2 + 4*q + 1),
 'variance_y': (-7/64*q^6 - 3/4*q^5 - 159/64*q^4 - 2*q^3 + 3/4*q^2 + 3/16*q + 1/8)/(-q^6 - 4*q^5 - 5*q^4 + 5*q^2 + 4*q + 1)}

In [16]:
qmin

8

In [17]:
results(2)

{'covariance': -17/108,
 'expectation_x': 1/6,
 'expectation_y': 1/6,
 'variance_x': 37/108,
 'variance_y': 37/108}

In [18]:
results(4)

{'covariance': -5513/75000,
 'expectation_x': 7/50,
 'expectation_y': 7/50,
 'variance_x': 15737/75000,
 'variance_y': 15737/75000}

In [19]:
results(6)

{'covariance': -9299/168070,
 'expectation_x': 13/98,
 'expectation_y': 13/98,
 'variance_x': 29117/168070,
 'variance_y': 29117/168070}

Checking that variance and determinant of variance-covariance matrix are non-zero:

In [20]:
RQ_single = QQ[q]
RQ_single(results(q)['variance_x'].numerator()).real_roots(), RQ_single(results(q)['variance_x'].denominator()).real_roots()

([-1.53633336552046, 0.487762442216091], [-1.00000000000000, 1.00000000000000])

In [21]:
RQ_single((results(q)['variance_x']*results(q)['variance_y'] - results(q)['covariance']^2).numerator()).real_roots(), \
RQ_single((results(q)['variance_x']*results(q)['variance_y'] - results(q)['covariance']^2).denominator()).real_roots()

([-5.37228132326901,
  -1.52829503621145,
  0.000000000000000,
  0.372281323269014,
  0.903720176681811],
 [-1.00000000000000, 1.00000000000000])

In [22]:
all(results(qq)[what] == results(q)[what].subs(q=qq)
    for qq in (6,)
    for what in ('covariance', 'expectation_x', 'expectation_y', 'variance_x', 'variance_y'))

True